# 1 Dependencies

In [10]:
from datetime import date, datetime
from typing import Optional, List
from sqlalchemy import create_engine, text, Integer, BigInteger, String, ForeignKey, Date, CheckConstraint, select, func, case, text, cast, DDL, event
from sqlalchemy.engine import URL
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session
from sqlalchemy.exc import ProgrammingError
import pandas as pd

# 2 Create a database

In [11]:
server_name = "Alstation"
database_name = "bookstore"
username = "sa"
password = "D@tabases"

def get_engine(server: str, username: str, password: str, database: Optional[str] = None):
    if database:
        conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    else:
        conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};UID={username};PWD={password}"
    url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_str})
    return create_engine(url)

# Connect to the server (without specifying database)
server_engine = get_engine(server_name, username, password)

try:
    with server_engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
        conn.execute(text(f"CREATE DATABASE {database_name}"))
except ProgrammingError as err:
    err_string = str(err)
    if "'bookstore' already exists" in err_string:
        print(f"A database named \"{database_name}\" already exists.")
    else:
        print(err)

# Connect to the database
engine = get_engine(server_name, username, password, database_name)
print(f"\nConnection to {database_name} successful!")


Connection to bookstore successful!


# 3 Create tables

In [12]:
class Base(DeclarativeBase):
    pass

class Authors(Base):
    __tablename__ = "authors"

    id: Mapped[int] = mapped_column(primary_key=True)
    first_name: Mapped[str] = mapped_column(String(30))
    last_name: Mapped[str] = mapped_column(String(30))
    birth_date: Mapped[Optional[date]] = mapped_column(Date)

    book_authors: Mapped[List["BookAuthors"]] = relationship(back_populates="authors", cascade="all, delete-orphan")  # deleting an author deletes it and related items in the junction table

class Books(Base):
    __tablename__ = "books"
    __table_args__ = (
        CheckConstraint("price > 0", name="check_price_positive"),
        CheckConstraint("isbn between 1000000000000 and 9999999999999", name="check_isbn_length")
    )

    isbn: Mapped[int] = mapped_column(BigInteger, primary_key=True, autoincrement=False)
    title: Mapped[str] = mapped_column(String(100))
    language: Mapped[Optional[str]] = mapped_column(String(30))
    price: Mapped[int] = mapped_column(Integer)
    publication_date: Mapped[Optional[date]] = mapped_column(Date)

    inventory: Mapped[List["Inventory"]] = relationship(back_populates="books", cascade="all, delete-orphan")  # deleting a book deletes its inventory
    book_authors: Mapped[List["BookAuthors"]] = relationship(back_populates="books", cascade="all, delete-orphan")  # deleting a book deletes it and related items in the junction table
    order_details: Mapped["OrderDetails"] = relationship(back_populates="books")
    reviews: Mapped[List["Reviews"]] = relationship(back_populates="books", cascade="all, delete-orphan")  # deleting a book deletes its reviews

class Stores(Base):
    __tablename__ = "stores"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(50))
    address: Mapped[Optional[str]] = mapped_column(String(100))

    inventory: Mapped[List["Inventory"]] = relationship(back_populates="stores", cascade="all, delete-orphan")  # deleting a store deletes it's inventory

class Inventory(Base):
    __tablename__ = "inventory"

    store_id: Mapped[int] = mapped_column(ForeignKey("stores.id"), primary_key=True)
    isbn: Mapped[int] = mapped_column(BigInteger, ForeignKey("books.isbn"), primary_key=True)
    amount: Mapped[int] = mapped_column(Integer, default=0)

    books: Mapped["Books"] = relationship(back_populates="inventory")
    stores: Mapped["Stores"] = relationship(back_populates="inventory")

class BookAuthors(Base):
    __tablename__ = "book_authors"

    isbn: Mapped[int] = mapped_column(BigInteger, ForeignKey("books.isbn"), primary_key=True)
    author_id: Mapped[int] = mapped_column(ForeignKey("authors.id"), primary_key=True)
    role: Mapped[Optional[str]] = mapped_column(String(20))

    books: Mapped["Books"] = relationship(back_populates="book_authors")
    authors: Mapped["Authors"] = relationship(back_populates="book_authors")

class Customers(Base):
    __tablename__ = "customers"
    __table_args__ = (
        CheckConstraint("email like '%@%.%'", name="check_customers_email_format"),
    )

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    first_name: Mapped[str] = mapped_column(String(30))
    last_name: Mapped[str] = mapped_column(String(30))
    email: Mapped[str] = mapped_column(String(80))
    membership_level: Mapped[str] = mapped_column(String(8))

    orders: Mapped[List["Orders"]] = relationship(back_populates="customers", cascade="all, delete-orphan")  # deleting a customer deletes their orders
    reviews: Mapped[List["Reviews"]] = relationship(back_populates="customers", cascade="all, delete-orphan")  # deleting a customer deletes their reviews

class Orders(Base):
    __tablename__ = "orders"

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    customer_id: Mapped[int] = mapped_column(ForeignKey("customers.id"))
    order_date: Mapped[date] = mapped_column(Date)

    customers: Mapped["Customers"] = relationship(back_populates="orders")
    order_details: Mapped[List["OrderDetails"]] = relationship(back_populates="orders", cascade="all, delete-orphan")  # deleting an order deletes its order details

class OrderDetails(Base):
    __tablename__ = "order_details"

    order_id: Mapped[int] = mapped_column(ForeignKey("orders.id"), primary_key=True)
    isbn: Mapped[int] = mapped_column(BigInteger, ForeignKey("books.isbn"), primary_key=True)
    quantity: Mapped[int] = mapped_column(Integer)

    orders: Mapped["Orders"] = relationship(back_populates="order_details")
    books: Mapped["Books"] = relationship(back_populates="order_details")

class Reviews(Base):
    __tablename__ = "reviews"

    isbn: Mapped[int] = mapped_column(BigInteger, ForeignKey("books.isbn"), primary_key=True)
    customer_id: Mapped[int] = mapped_column(ForeignKey("customers.id"), primary_key=True)
    rating: Mapped[int] = mapped_column(Integer)

    books: Mapped["Books"] = relationship(back_populates="reviews")
    customers: Mapped["Customers"] = relationship(back_populates="reviews")


Base.metadata.create_all(engine)

# 3 Uploading data

In [13]:
table_names = [table for table in Base.metadata.tables]
csv_paths = [f"data/{table}.csv" for table in table_names]
column_names = [[col.name for col in cls.__table__.columns] for cls in Base.__subclasses__()]
identity_tables = ["authors", "stores", "customers", "orders"]

for table, path, cols in zip(table_names, csv_paths, column_names):
    df = pd.read_csv(path, names=cols, header=0)
    if table in identity_tables and "id" in df.columns:
        df = df.drop(columns=["id"])
    df.to_sql(name=table, con=engine, if_exists="append", index=False)

In [14]:
# engine.dispose()  # now the database is no longer in use

# 4 Author overview

## SQLAlchemy Core "kinda"

In [15]:
session = Session(engine)
stmt = text("""
        select 
            a.id,
            concat(a.first_name, ' ', a.last_name) as name,
            cast(datediff(year, a.birth_date, cast(getdate() AS date)) as nvarchar(max)) + ' years' as age,
            cast(count(distinct b.isbn) as nvarchar(max)) + ' copies' as titles,
            cast(sum(i.amount * b.price) as nvarchar(max)) + ' SEK' as inventory
        from 
            authors a 
        join
            book_authors ba on a.id = ba.author_id
        join
            books b on ba.isbn = b.isbn
        join 
            inventory i on b.isbn = i.isbn
        group by
            a.id,
            a.first_name,
            a.last_name,
            a.birth_date
    """)

result = session.execute(stmt)
df = pd.DataFrame(result.fetchall(), columns=result.keys())
df

,id,name,age,titles,inventory
0,1,Brandon Sanderson,50 years,11 copies,158769 SEK
1,2,G.R.R. Martin,77 years,7 copies,91622 SEK
2,3,Suzanne Collins,63 years,5 copies,52523 SEK
3,4,Liu Cixin,62 years,3 copies,33722 SEK
4,5,Holly Black,54 years,3 copies,61110 SEK
5,6,Henrik Larsson,47 years,3 copies,20777 SEK
6,7,Robert Jordan,77 years,14 copies,151610 SEK
7,8,Gardner Dozois,78 years,2 copies,22155 SEK
8,9,J.R.R. Tolkien,133 years,5 copies,38306 SEK
9,10,Christopher Tolkien,101 years,1 copies,9487 SEK


## ORM attempt

In [16]:
author_overview_def = DDL("""
    create or alter view author_overview as
    select 
        a.id,
        concat(a.first_name, ' ', a.last_name) as name,
        cast(datediff(year, a.birth_date, cast(getdate() AS date)) as nvarchar(max)) + ' years' as age,
        cast(count(distinct b.isbn) as nvarchar(max)) + ' copies' as titles,
        cast(sum(i.amount * b.price) as nvarchar(max)) + ' SEK' as inventory
    from 
        authors a 
    join
        book_authors ba on a.id = ba.author_id
    join
        books b on ba.isbn = b.isbn
    join 
        inventory i on b.isbn = i.isbn
    group by
        a.id,
        a.first_name,
        a.last_name,
        a.birth_date
""")

event.listen(Base.metadata, "after_create", author_overview_def)

class AuthorOverview(Base):
    __tablename__ = "author_overview"
    __table_args__ = {"info": {"is_view": True}}

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(64))
    age: Mapped[str] = mapped_column(String(9))
    titles: Mapped[str] = mapped_column(String(14))
    inventory: Mapped[str] = mapped_column(String(12))

session = Session(engine)
results = session.query(AuthorOverview).all()
for author in results:
    print(f"""
        {str(author.name).ljust(15)}
        {str(author.age).ljust(15)}
        {str(author.titles).ljust(15)}
        {str(author.inventory).ljust(15)}
    """)

ProgrammingError: (pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'author_overview'. (208) (SQLExecDirectW)")
[SQL: SELECT author_overview.id AS author_overview_id, author_overview.name AS author_overview_name, author_overview.age AS author_overview_age, author_overview.titles AS author_overview_titles, author_overview.inventory AS author_overview_inventory 
FROM author_overview]
(Background on this error at: https://sqlalche.me/e/20/f405)